In [1]:
from openai import OpenAI
import chromadb
import re
import json
import uuid


In [2]:
client = OpenAI(api_key="YOUR_API_KEY")
embed_model = "text-embedding-3-small"

# Vector Store

In [3]:
# Load your parsed doc JSON
with open("data_processed/parsed_pdf_docs.json") as f:
    pages = json.load(f)

In [4]:
# Combine both signals
def hybrid_text(page):
    # Tune merging as you wish:
    return f"PAGE OCR TEXT:\n{page['raw_text']}\n\nPAGE DESCRIPTION:\n{page['img_desc']}"

In [5]:
texts = [hybrid_text(p) for p in pages]

In [6]:
metadatas = [
    {
        "company": p["company"],
        "year": p["year"],
        "pdf_name": p["pdf_name"],
        "page_number": p["page_number"]
    }
    for p in pages
]

In [7]:
# Get embeddings
embeddings = []
for i in range(0, len(texts), 50):
    batch = texts[i:i+50]
    response = client.embeddings.create(model=embed_model, input=batch)
    embeddings.extend([e.embedding for e in response.data])

In [8]:
# Index in Chroma
import chromadb
# chroma_client = chromadb.Client()
chroma_client = chromadb.PersistentClient(path='vector_store')
coll = chroma_client.get_or_create_collection("hybrid_financial_chunks")

In [9]:
# for text, meta, emb in zip(texts, metadatas, embeddings):
#     coll.add(embeddings=[emb], metadatas=[meta], documents=[text])

for text, meta, emb in zip(texts, metadatas, embeddings):
    unique_id = str(uuid.uuid4())
    coll.add(
        ids=[unique_id],
        embeddings=[emb],
        metadatas=[meta],
        documents=[text]
    )

# Retrieval

In [10]:
chroma_client = chromadb.PersistentClient(path='vector_store')
collection = chroma_client.get_collection("hybrid_financial_chunks")

In [11]:
def retrieve_financial_info(subquery, company_name, year):
    subquery = subquery.lower()
    company = company_name.lower()
    top_k=3
    embed_resp = client.embeddings.create(model=embed_model, input=[subquery])
    query_emb = embed_resp.data[0].embedding
    
    results = collection.query(
        query_embeddings=[query_emb],  # semantic
        query_texts=[subquery],           # keyword
        n_results=top_k,
        where={
            "$and": [
                {"company": company_name},
                {"year": year}
            ]
        }
    )
    
    docs = results['documents'][0]
    metas = results['metadatas'][0]
    outs = []
    for text, meta in zip(docs, metas):
        outs.append({
            "text": text,
            "page_number": meta.get("page_number"),
            "pdf_name": meta.get("pdf_name"),
            "company": meta.get("company"),
            "year": meta.get("year")
        })
    return outs

In [12]:
results = retrieve_financial_info(
    subquery="net income", 
    company_name="google", 
    year="2022"
)

In [13]:
results

[{'text': "PAGE OCR TEXT:\nTable of Contents\nAlphabet Inc.\nBasic net income per share:\nNumerator\nAllocation of undistributed earnings\nDenominator\nNumber of shares used in per share computation\nBasic net income per share\nDiluted net income per share:\nNumerator\nAllocation of undistributed earnings for basic computation\nReallocation of undistributed earnings as a result of conversion of Class B to Class A\nshares\nReallocation of undistributed earnings\nAllocation of undistributed earnings\nDenominator\nNumber of shares used in basic computation\nWeighted-average effect of dilutive securities\nAdd:\nConversion of Class B to Class A shares outstanding\nRestricted stock units and other contingently issuable shares\nNumber of shares used in per share computation\nDiluted net income per share\nBasic net income per share:\nNumerator\nAllocation of undistributed earnings\nDenominator\nNumber of shares used in per share computation\nBasic net income per share\nDiluted net income per s

# Agent

In [14]:
retrieval_function = {
    "name": "retrieve_financial_info",
    "description": "Retrieve relevant documents from annual reports given the company name, year, and subquery.",
    "parameters": {
        "type": "object",
        "properties": {
            "company_name": {
                "type": "string",
                "description": "The company name",
                "enum": ["google", "microsoft", "nvidia"] 
            },
            "year": {
                "type": "string",
                "description": "The year of the report",
                "enum": ["2022", "2023", "2024"]  
            },
            "subquery": {"type": "string"}
        },
        "required": ["company", "year", "query"]
    }
}


In [15]:
system_prompt = \
"""
You are a financial analyst assistant specializing in the finances of three companies: Google, Microsoft, and Nvidia.
Your task is to answer user queries about these companies using only reliable, retrieved information.

To find relevant information, use the retrieve_financial_info function. For each function call, you must specify:
1) A subquery based on the user’s main question that describes the key information you want to extract,
2) The company name,
3) And the year.

When answering, use only the retrieved information as context.
If you do not have any document for reference, reply: "I don't have relevant information to answer this query."

For every answer, always provide:
1) "answer": Your direct answer to the user's query, in clear, concise language.
2) "reasoning": A brief explanation of how you arrived at your answer, referencing the context you used.
3) "sources": A list of sources you used, with each source including:
    a) company
    b) year
    c) pdf_name
    d) page_number
    e) a short excerpt from the document used.

Respond only in the following JSON format: 
```
{
  "answer": "<your answer here>",
  "reasoning": "<your reasoning here>",
  "sources": [
    {
      "company": "<company>",
      "year": "<year>",
      "page": <page_number>,
      "excerpt": "<relevant excerpt from the retrieved document>"
    }
  ]
}
```

If you do not have any document for reference, output:
```
{ "answer": "I don't have relevant information to answer this query." }
```
"""

In [16]:
def get_llm_response(messages):
    tools = [retrieval_function]
    return client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            functions=tools,
            function_call="auto",
            max_tokens=700,
            temperature=0
        )

def get_tool_calls(response):
    tool_calls = []
    msg = response.choices[0].message
    if hasattr(msg, "tool_calls") and msg.tool_calls:
        tool_calls = [(call, 'tool') for call in msg.tool_calls]
    elif hasattr(msg, "function_call") and msg.function_call:
        tool_calls = [(msg.function_call, 'function')]
    return tool_calls


def get_tool_response(tool_call):
    call, role = tool_call
    func_args = json.loads(call.arguments)
    func_name = call.name
    sub_query_text = f"{func_args['company_name']} {func_args['subquery']} {func_args['year']}"
    print(f'Running tool `{func_name}` with `{func_args}`\nSubquery: {sub_query_text}')
    
    tool_response = {}
    if func_name == 'retrieve_financial_info':
        tool_response = retrieve_financial_info(**func_args)

    tool_call_id = getattr(call, "id", None)
    function_message = {
                "role": role,           
                "name": func_name,
                "tool_call_id": tool_call_id,
                "content": json.dumps(tool_response),
            }
    return function_message, sub_query_text


def generate_answer(query):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query}
    ]
    sub_queries = []
    while True:
        response = get_llm_response(messages)
        llm_message = response.choices[0].message  
        messages.append(llm_message)
        tool_calls = get_tool_calls(response)
        if tool_calls:
            for tool_call in tool_calls:
                function_message, sub_query_text = get_tool_response(tool_call)
                sub_queries.append(sub_query_text)
                # print(f'Function Message: {function_message}')
                messages.append(function_message)
            continue
        else:
            return llm_message, sub_queries


def parse_llm_response_to_dict(llm_response):
    content = llm_response.content
    content = content.strip()
    if content.startswith("```json"):
        content = content[len("```json"):].strip("` \n")
    elif content.startswith("```"):
        content = content.strip("` \n")
    try:
        return json.loads(content)
    except Exception as e:
        print(f"Failed to parse JSON: {e}\nContent was:\n{content}")
        return None

def get_output_json(query):
    llm_message, sub_queries = generate_answer(query)
    llm_res_dict = parse_llm_response_to_dict(llm_message)
    if llm_res_dict:
        llm_res_dict['subqueries'] = sub_queries
        return llm_res_dict
    else:
        ""

In [17]:
response = get_output_json('Which company had the highest operating margin in 2023?')

Running tool `retrieve_financial_info` with `{'company_name': 'google', 'year': '2023', 'subquery': 'operating margin'}`
Subquery: google operating margin 2023
Running tool `retrieve_financial_info` with `{'company_name': 'microsoft', 'year': '2023', 'subquery': 'operating margin'}`
Subquery: microsoft operating margin 2023
Running tool `retrieve_financial_info` with `{'company_name': 'nvidia', 'year': '2023', 'subquery': 'operating margin'}`
Subquery: nvidia operating margin 2023


In [18]:
response

{'answer': 'In 2023, Microsoft had the highest operating margin among the three companies.',
 'reasoning': "Based on the retrieved information, Microsoft's operating income for 2023 was $88,523 million, which represents a 6% increase from the previous year. Nvidia's gross margin decreased to 56.9% in 2023, and Google's total income from operations was $84,293 million. The specific operating margin percentages were not provided for all companies, but the available data suggests that Microsoft's financial performance was strong, with significant growth in operating income.",
 'sources': [{'company': 'microsoft',
   'year': '2023',
   'page': 51,
   'excerpt': 'GAAP operating income for 2023 is $88,523 million, a 6% increase from $83,383 million in 2022.'},
  {'company': 'nvidia',
   'year': '2023',
   'page': 49,
   'excerpt': 'Gross margin was 56.9% and 64.9% for fiscal years 2023 and 2022, respectively.'},
  {'company': 'google',
   'year': '2023',
   'page': 42,
   'excerpt': 'Overall